In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

In [3]:
data1 = sio.loadmat('ex8_movies.mat')
R = data1['R']
Y = data1['Y']

In [4]:
Y[0, R[0, :]==1].mean()

3.8783185840707963

In [5]:
data2 = sio.loadmat('ex8_movieParams.mat')
theta = data2['Theta']
X = data2['X']
nf = data2['num_features'][0,0]
nu = data2['num_users'][0,0]
nm = data2['num_movies'][0,0]

In [6]:
def computeCost(params, Y, R, lamda, nf, nu, nm):
    X = params[:nm*nf].reshape(nm, nf)
    theta = params[nm*nf:].reshape(nu, nf)
    X_grad = np.zeros(X.shape)
    theta_grad = np.zeros(theta.shape)
    
    J_temp = (X.dot(theta.T)-Y)**2
    J = (J_temp[R==1].sum())/2 + lamda/2*(theta**2).sum() \
        + lamda/2*(X**2).sum()
    
    return J
def computeGrad(params, Y, R, lamda, nf, nu, nm):
    X = params[:nm*nf].reshape(nm, nf)
    theta = params[nm*nf:].reshape(nu, nf)
    X_grad = np.zeros(X.shape)
    theta_grad = np.zeros(theta.shape)
    
    X_grad = ((X.dot(theta.T)-Y)*R).dot(theta) + lamda*X
    theta_grad = ((X.dot(theta.T)-Y)*R).T.dot(X) + lamda*theta
    return np.r_[X_grad.flatten(), theta_grad.flatten()]

In [7]:
J = computeCost(np.r_[X.flatten(), theta.flatten()], Y, R, 0, nf, nu, nm)

In [8]:
#test
nut, nmt, nft = 4, 5, 3
X1 = X[0:nmt,0:nft]
theta1 = theta[0:nut,0:nft]
Y1 = Y[0:nmt,0:nut]
R1 = R[0:nmt,0:nut]

#check the cost
J = computeCost(np.r_[X1.flatten(), theta1.flatten()],Y1,R1,0, nft, nut, nmt)
print("lamda = 0,this value should be about 22.22\n",J)

J2= computeCost(np.r_[X1.flatten(), theta1.flatten()],Y1,R1,1.5, nft, nut, nmt)
print("lamda = 1.5,this value should be about 31.34\n",J2)

lamda = 0,this value should be about 22.22
 22.2246037257
lamda = 1.5,this value should be about 31.34
 31.3440562443


In [9]:
#check gradient
def checkCost(lamda=0):
    np.random.seed(44)
    X_t = np.random.random((4, 3))
    theta_t = np.random.random((5, 3))
    Y = X_t.dot(theta_t.T)
    Y[np.random.rand(*Y.shape)>0.5] = 0
    R = np.zeros_like(Y)
    R[Y!=0] = 1 
    
    X = np.random.randn(*X_t.shape)
    theta = np.random.randn(*theta_t.shape)
    nf = 3
    nu = 5
    nm = 4
    #grad
    theta_grad = computeGrad(np.r_[X.flatten(), theta.flatten()], Y, R, lamda, nf, nu, nm)[nm*nf:]
    J_test = computeCost(np.r_[X.flatten(), theta.flatten()], Y, R, 1.5, nf, nu, nm)
    print(J_test)
    #num grad
    numgrad = np.zeros(theta_t.flatten().shape)
    perturb = np.zeros(theta_t.flatten().shape)
    e = 1e-4
    J = lambda p: computeCost(p, Y, R, lamda, nf, nu, nm)
    for i in range(theta.size):
        perturb[i:i+1] = e
        loss1 = J(np.r_[X.flatten(), theta.flatten()-perturb])
        loss2 = J(np.r_[X.flatten(), theta.flatten()+perturb])
        numgrad[i:i+1] = (loss2-loss1)/(2*e)
        perturb[i:i+1] = 0
    dice = np.linalg.norm(numgrad.flatten()-theta_grad)/np.linalg.norm(numgrad.flatten()+theta_grad)
    print(dice)
    return numgrad, theta_grad

In [10]:
checkCost()

22.9010753253
9.59267737147e-13


(array([-0.05325967, -0.12562019, -0.62648596,  0.02506604, -0.69649836,
         0.40062249,  0.35024053,  1.24076044,  4.02004181,  0.15252736,
         1.42006358,  1.5635142 ,  0.26312911,  1.67984048,  2.86109063]),
 array([-0.05325967, -0.12562019, -0.62648596,  0.02506604, -0.69649836,
         0.40062249,  0.35024053,  1.24076044,  4.02004181,  0.15252736,
         1.42006358,  1.5635142 ,  0.26312911,  1.67984048,  2.86109063]))

In [11]:
def loadMovies(file):
    with open(file, encoding='utf-8', errors='ignore') as f:
        movielist = []
        for line in f:
            line_sp = line.strip().split(' ')
            name = ''
            for i in range(len(line_sp)):
                if i == 0:
                    continue
                name = name + line_sp[i] + ' '
            movielist.append(name)   
    return movielist
movieList = loadMovies('movie_ids.txt')

In [12]:
my_ratings = np.zeros((nm, 1))
my_ratings[0] = 4
my_ratings[97] = 2
my_ratings[6] = 3
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63]= 5
my_ratings[65]= 3
my_ratings[68] = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354]= 5
for i in range(nm):
    if my_ratings[i]>0:
        print("Rated %d for %s" %(my_ratings[i],movieList[i]))

Rated 4 for Toy Story (1995) 
Rated 3 for Twelve Monkeys (1995) 
Rated 5 for Usual Suspects, The (1995) 
Rated 4 for Outbreak (1995) 
Rated 5 for Shawshank Redemption, The (1994) 
Rated 3 for While You Were Sleeping (1995) 
Rated 5 for Forrest Gump (1994) 
Rated 2 for Silence of the Lambs, The (1991) 
Rated 4 for Alien (1979) 
Rated 5 for Die Hard 2 (1990) 
Rated 5 for Sphere (1998) 


In [13]:
data3 = sio.loadmat('ex8_movies.mat')
R3 = data3['R']
Y3 = data3['Y']
print(R3.shape, Y3.shape)

(1682, 943) (1682, 943)


In [14]:
Ymy = np.c_[(my_ratings,Y3)]
Rmy = np.c_[(my_ratings!=0,R3)]
print(Ymy.shape, Rmy.shape)

(1682, 944) (1682, 944)


In [15]:
def norm(Y, R):
    m, n = Y.shape
    Ymean = np.zeros((m, 1))
    Ynorm = np.zeros(Y.shape)
    for i in range(m):
        idx = np.where(R[i:i+1, :]==1)
        Ymean[i:i+1, :] = Y[idx].mean()
        Ynorm[i:i+1, :] = Y[i:i+1, :]-Ymean[i:i+1, :]
    return Ynorm, Ymean
Ynorm, Ymean = norm(Ymy, Rmy)
print(Ynorm.shape, Ymean.shape)

(1682, 944) (1682, 1)


In [16]:
nf3 = 10
nm3, nu3 = Ynorm.shape


In [42]:
X3 = np.random.randn(nm3, nf3)
theta3 = np.random.randn(nu3, nf3)
import scipy.optimize as op
lamda = 1.5
movie_params = op.fmin_cg(f=computeCost, fprime=computeGrad, x0=np.r_[X3.flatten(), theta3.flatten()], 
          args=(Ynorm, Rmy, lamda, nf3, nu3, nm3), maxiter=100)#, retall=True)
print(movie_params)

         Current function value: 30112.348192
         Iterations: 100
         Function evaluations: 155
         Gradient evaluations: 155
[-0.23747808  0.25336628  0.03190838 ...,  0.79804325 -1.13657142
 -1.1108632 ]


In [43]:
bestX = movie_params[:nm3*nf3].reshape(nm3, nf3)
bestTheta = movie_params[nm3*nf3:].reshape(nu3, nf3)

In [44]:
score = bestX.dot(bestTheta.T) + Ymean
#只有第一行是新用户的分数
my_score = score[:,0]  #line 1 is my scoce
print(score.shape)
print(my_score[:5])
#排序，推荐最高的分数的电影给新用户
sort_index = my_score.argsort()
favorite = 10
for i in range(favorite):
    print("favorite %d score is %d,for:%s" \
          %(i+1,my_score[sort_index[-(i+1)]],movieList[sort_index[-(i+1)]]))

(1682, 944)
[ 3.86314788  3.19597245  4.00315497  3.85730101  3.19329557]
favorite 1 score is 5,for:Bye Bye, Love (1995) 
favorite 2 score is 5,for:Higher Learning (1995) 
favorite 3 score is 4,for:Witness (1985) 
favorite 4 score is 4,for:Fifth Element, The (1997) 
favorite 5 score is 4,for:Nobody Loves Me (Keiner liebt mich) (1994) 
favorite 6 score is 4,for:I Don't Want to Talk About It (De eso no se habla) (1993) 
favorite 7 score is 4,for:Return of the Jedi (1983) 
favorite 8 score is 4,for:Star Wars (1977) 
favorite 9 score is 4,for:Mad Dog Time (1996) 
favorite 10 score is 4,for:Everest (1998) 
